In [19]:
def fitnessFunction(chromosome,network):
    mat = network['mat']
    fitness = 0.0
    for i in range(len(chromosome) - 1):
        fitness += mat[chromosome[i]][chromosome[i+1]]
    return fitness + mat[chromosome[len(chromosome) - 1]][chromosome[0]]


In [20]:
from random import randint, seed,shuffle


def generateARandomPermutation(n):
    perm = [i for i in range(n)]
    shuffle(perm)
    return perm

# permutation-based representation
class Chromosome:
    def __init__(self, problParam = None):
        self.__problParam = problParam  #problParam has to store the number of nodes/cities
        self.__repres = generateARandomPermutation(self.__problParam['noNodes'])
        self.__fitness = 0.0
    
    @property
    def repres(self):
        return self.__repres 
    
    @property
    def fitness(self):
        return self.__fitness 
    
    @repres.setter
    def repres(self, l = []):
        self.__repres = l 
    
    @fitness.setter 
    def fitness(self, fit = 0.0):
        self.__fitness = fit 
    
    def crossover(self, c):
        # order XO
        pos1 = randint(-1, self.__problParam['noNodes'] - 1)
        pos2 = randint(-1, self.__problParam['noNodes'] - 1)
        if (pos2 < pos1):
            pos1, pos2 = pos2, pos1 
        k = 0
        newrepres = self.__repres[pos1 : pos2]
        for el in c.__repres[pos2:] +c.__repres[:pos2]:
            if (el not in newrepres):
                if (len(newrepres) < self.__problParam['noNodes'] - pos1):
                    newrepres.append(el)
                else:
                    newrepres.insert(k, el)
                    k += 1

        offspring = Chromosome(self.__problParam)
        offspring.repres = newrepres
        return offspring

    def doubleMove(self,c,value):
        for i in range(len(c.repres)):
            if self.__repres[i] == value:
                value = c.__repres[i]
                break
        for i in range(len(c.repres)):
            if self.__repres[i] == value:
                value = c.__repres[i]
                break
        return value

    def simpleMove(self,c,value):
        for i in range(len(c.__repres)):
            if self.__repres[i] == value:
                value = c.__repres[i]
                return value

    def find_first_not_in(self,off):
        for i in range(len(self.__repres)):
            if self.__repres[i] not in off:
                return self.__repres[i]

    def crossover2(self,c):
        represOff1= []
        represOff2= []
        represOff1.append(c.__repres[0])
        print(c.__repres)
        print(self.__repres)
        represOff2.append(self.doubleMove(c,represOff1[0]))
        print(represOff1)
        print(represOff2)
        while(represOff2[-1] != self.__repres[0]):
            represOff1.append(self.simpleMove(c,represOff2[-1]))
            represOff2.append(self.doubleMove(c,represOff1[-1]))
        if len(represOff1) < self.__problParam['noNodes']:
            represOff1.append(self.find_first_not_in(represOff2))
            represOff1.append(self.doubleMove(c,represOff1[-1]))
            while len(represOff2) <= self.__problParam['noNodes']:
                represOff1.append(self.simpleMove(c,represOff2[-1]))
                represOff2.append(self.doubleMove(c,represOff1[-1]))
        offspring1 = Chromosome(self.__problParam)
        offspring1.repres = represOff1
        offspring2 = Chromosome(self.__problParam)
        offspring2.repres = represOff2
        if fitnessFunction(offspring1.repres, self.__problParam) < fitnessFunction(offspring2.repres, self.__problParam):
            return offspring1
        return offspring2

        

    
    def mutation(self):
        # insert mutation
        pos1 = randint(0, self.__problParam['noNodes'] - 1)
        pos2 = randint(0, self.__problParam['noNodes'] - 1)
        if (pos2 < pos1):
            pos1, pos2 = pos2, pos1
        el = self.__repres[pos2]
        del self.__repres[pos2]
        self.__repres.insert(pos1 + 1, el)
        
    def __str__(self):
        return "\nChromo: " + str(self.__repres) + " has fit: " + str(self.__fitness)
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, c):
        return self.__repres == c.__repres

In [21]:
def readNet(fileName):
    f = open(fileName, "r")
    net = {}
    n = int(f.readline())
    net['noNodes'] = n
    mat = []
    for i in range(n):
        mat.append([])
        line = f.readline()
        elems = line.split(",")
        for j in range(n):
            mat[-1].append(int(elems[j]))
    net["mat"] = mat 
    degrees = []
    noEdges = 0
    for i in range(n):
        d = 0
        for j in range(n):
            if (mat[i][j] == 1):
                d += 1
            if (j > i):
                noEdges += mat[i][j]
        degrees.append(d)
    net["noEdges"] = noEdges
    net["degrees"] = degrees
    f.close()
    return net

In [22]:
def euclideanDistance(a, b):
    return sqrt((b[0]-a[0])*(b[0]-a[0])+(b[1]-a[1])*(b[1]-a[1]))


def read_berlin(filePath):
    mat = {}
    f = open(filePath, "r")
    words = f.readline().split(' ')
    while words[0] != "DIMENSION":
        words = f.readline().split(' ')
    mat['noNodes'] = int(words[2])
    f.readline()
    f.readline()
    lines = []
    for i in range(0, mat['noNodes']):
        words = f.readline().split(" ")
        lines.append((float(words[1]), float(words[2])))

    mat['mat'] = []
    for i in range(0, mat['noNodes']):
        mat['mat'].append([])
        for j in range(0, mat['noNodes']):
            mat['mat'][i].append(0.0)
    for i in range(0, mat['noNodes'] - 1):
        for j in range(i + 1, mat['noNodes']):
            mat['mat'][i][i] = 0.0
            mat['mat'][j][i] = mat['mat'][i][j] = euclideanDistance(lines[i], lines[j])
    return mat

In [23]:
from random import *

class GA:
    def __init__(self, param = None, problParam = None):
        self.__param = param
        self.__problParam = problParam
        self.__population = []
        
    @property
    def population(self):
        return self.__population
    
    def initialisation(self):
        for _ in range(0, self.__param['popSize']):
            c = Chromosome(self.__problParam['network'])
            self.__population.append(c)
    
    def evaluation(self):
        for c in self.__population:
            c.fitness = self.__problParam['function'](c.repres,self.__problParam['network'])
            
    def bestChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness < best.fitness):
                best = c
        return best
        
    def worstChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness > best.fitness):
                best = c
        return best

    def selection(self):
        sumfitness =0 
        for c in self.__population:
            sumfitness += c.fitness
        rel_fitness=[]
        for c in self.__population:
            rel_fitness.append(c.fitness/sumfitness) 
        probabilities = [sum(rel_fitness[:i + 1]) for i in range(len(rel_fitness))]

        r = random()
        for i in range(self.__param['popSize']):
            if r <= probabilities[i]:
                return i
        
    def selection1(self):
        pos1 = randint(0, self.__param['popSize'] - 1)
        pos2 = randint(0, self.__param['popSize'] - 1)
        if (self.__population[pos1].fitness < self.__population[pos2].fitness):
            return pos1
        else:
            return pos2 

    
    def oneGeneration(self):
        finalPop = []
        for _ in range(self.__param['popSize']):
            p1 = self.__population[self.selection1()]
            p2 = self.__population[self.selection1()]
            off = p1.crossover(p2)
            off.mutation()
            self.__population.append(off)

        self.evaluation()
        self.__population.sort(key=lambda x: x.fitness, reverse=False)
        for i in range(self.__param['popSize']):
            finalPop.append(self.__population[i])
        self.__population = finalPop
        self.evaluation()

    def oneGenerationElitism(self):
        newPop = [self.bestChromosome()]
        for _ in range(self.__param['popSize'] - 1):
            p1 = self.__population[self.selection1()]
            p2 = self.__population[self.selection1()]
            off = p1.crossover2(p2)
            off.mutation()
            newPop.append(off)
        self.__population = newPop
        self.evaluation()
        
    def oneGenerationSteadyState(self):
        for _ in range(self.__param['popSize']):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            off.fitness = self.__problParam['function'](off.repres,self.__problParam['network'])
            worst = self.worstChromosome()
            if (off.fitness < worst.fitness):
                worst = off

In [24]:
from random import seed 

def findClusters(net,popSize,noGen):
    seed(1)

    # initialise de GA parameters
    gaParam = {'popSize' : popSize, 'noGen' : noGen}
    # problem parameters
    problParam ={'network': net,'function':fitnessFunction}
    ga = GA(gaParam, problParam)
    ga.initialisation()
    ga.evaluation()
    
    for g in range(gaParam['noGen']):
        #logic alg
        ga.oneGeneration()
        bestChromo = ga.bestChromosome()

        print('Best solution in generation ' + str(g) + ' is: x = ' + str(bestChromo.repres) + ' f(x) = ' + str(bestChromo.fitness))

In [28]:
import os
import warnings
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt 
from math import sqrt

crtDir = os.getcwd()
filePathGML = os.path.join(crtDir,'net.in')
network = readNet(filePathGML)
bst = findClusters(network,300,1600)

, 7, 8] f(x) = 953.0
Best solution in generation 282 is: x = [15, 18, 19, 17, 16, 20, 21, 25, 22, 23, 24, 11, 12, 10, 9, 13, 14, 0, 1, 2, 3, 5, 4, 6, 7, 8] f(x) = 953.0
Best solution in generation 283 is: x = [15, 18, 19, 17, 16, 20, 21, 25, 22, 23, 24, 11, 12, 10, 9, 13, 14, 0, 1, 2, 3, 5, 4, 6, 7, 8] f(x) = 953.0
Best solution in generation 284 is: x = [15, 18, 19, 17, 16, 20, 21, 25, 22, 23, 24, 11, 12, 10, 9, 13, 14, 0, 1, 2, 3, 5, 4, 6, 7, 8] f(x) = 953.0
Best solution in generation 285 is: x = [15, 18, 19, 17, 16, 20, 21, 25, 22, 23, 24, 11, 12, 10, 9, 13, 14, 0, 1, 2, 3, 5, 4, 6, 7, 8] f(x) = 953.0
Best solution in generation 286 is: x = [15, 18, 19, 17, 16, 20, 21, 25, 22, 23, 24, 11, 12, 10, 9, 13, 14, 0, 1, 2, 3, 5, 4, 6, 7, 8] f(x) = 953.0
Best solution in generation 287 is: x = [15, 18, 19, 17, 16, 20, 21, 25, 22, 23, 24, 11, 12, 10, 9, 13, 14, 0, 1, 2, 3, 5, 4, 6, 7, 8] f(x) = 953.0
Best solution in generation 288 is: x = [15, 18, 19, 17, 16, 20, 21, 25, 22, 23, 24, 11, 12

KeyboardInterrupt: 